# Assignment #4

READ ALL INSTRUCTIONS CAREFULLY

This assignment is a puzzle. You have been given a data set with some semi-informative column headers but no additional information about it. Your goal is to investigate the data, understand as much about it as you can, and report back what you find.

You are free to use any and all of the techniques we have covered this semester: EDA, data wrangling, visualization (scatter plots, line plots, histograms, jointplots, etc.), descriptive statistics (mean, median, variance, covariance, etc.), regression / model fitting, loss functions (MSE, MAE), R^2, cross-validation, bootstrapping, gradient descent, etc... 

There are many different ways to approach this assignment and be successful. I recommend starting simple and getting slowly more sophisticated as needed.

## Collaborators

You must work in groups of 2-3. You cannot work on this assignment alone. Your partners cannot be anyone you have partnered with previously on an assignment and they cannot be in your group for the semester project.

**Names of all collaborators go here** (including yourself):
- Isaia Pacheco
- Alex Garrigan-Timm
- Noam Joseph

## Tools

You are not allowed to use LLMs (including the LLM results that pop up in Google search). You may use search online for ideas, use StackExchange, and I highly encourage using Python library documentation.

Hint: It may be useful to use the sklearn library for model fitting (and not to implement model fitting or optimization by hand).

## Submission Instructions

You must submit two files. One is your **Jupyter notebook** that you use to explore and analyze the data. The second file is a separate **PDF** reporting, presenting, and explaining your results with no code in it.

**Only one person** in your group should submit the assignment files on Canvas. Be sure everyone's names are listed under collaborators in both the notebook and the PDF so everyone gets credit!

## Potentially Helpful Documentation


https://scikit-learn.org/stable/api/sklearn.model_selection.html

https://scikit-learn.org/stable/modules/linear_model.html

https://scikit-learn.org/stable/api/sklearn.metrics.html

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

# Your Code Below

## Imports and CoW

In [2]:
# Add all of your import statements here. ONLY include the imports you actually use. There are useful tools in the scikit-learn library for regression and model fitting, but you are not required to use it. If you use it, you will need to install it into your virtual environment.
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sklearn as sk
# from sklearn.<something> import <something>
from sklearn.model_selection import train_test_split

In [3]:
# Enforce Copy-on-Write (CoW) semantics here
pd.options.mode.copy_on_write = True
pd.set_option('display.max_rows', 100)

## Project Folder Organization and File Reading

You must read in the given data file using relative paths, as shown in previous assignments. Organize your work to be in a project folder, with src and data subfolders. Place your solution notebook in the src folder, and the given data file in the data folder.

In [ ]:
# Read in the data file (using relative paths for reproducibility). It should be able to download your notebook, put it in the src folder of my project folder, and run it without changing anything.
def read_csv(file, **kwargs):
    cwd = Path.cwd()
    assignment2 = cwd.parent
    data = assignment2.joinpath('data')
    path = data.joinpath(file)
    if path.exists():
        print(f'Found the data file {path}')
    else:
        raise Exception('Could not find that file', path)
    return pd.read_csv(path, **kwargs)

#### mystery.csv appears to separate the columns with a pipe or "|"

In [ ]:
mystery_df = read_csv('mystery.csv', delimiter='|', float_precision='round_trip')

In [ ]:
seed = 4
mystery_df_train, mystery_df_test = train_test_split(mystery_df[:], train_size=0.79, random_state=seed)
mystery_df_eval, mystery_df_test = train_test_split(mystery_df_test, test_size=0.5, random_state=seed)
print(f'train: {mystery_df_train.shape}, test:{mystery_df_test.shape}, eval:{mystery_df_eval.shape}')

In [ ]:
mystery_df_train.value_counts().reset_index()

In [ ]:
before_describe = mystery_df_train.describe()

What does -999.0 mean? missing at random since all columns have -999?

In [ ]:
df = mystery_df_train[
        (mystery_df_train['x'] > -999.0) |
        (mystery_df_train['y'] > -999.0) |
        (mystery_df_train['z'] > -999.0) |
        (mystery_df_train['d1'] > -999.0) |
        (mystery_df_train['d2'] > -999.0) |
        (mystery_df_train['d3'] > -999.0) |
        (mystery_df_train['d4'] > -999.0)
]

In [ ]:
df.shape

In [ ]:
after_describe = df.describe()

In [ ]:
before_melted = before_describe.melt(ignore_index=False, var_name='Variable', value_name='df1_Value').reset_index().rename(columns={'index': 'Statistic'})
after_melted = after_describe.melt(ignore_index=False, var_name='Variable', value_name='df2_Value').reset_index().rename(columns={'index': 'Statistic'})
comparison_df = pd.merge(before_melted, after_melted, on=['Statistic', 'Variable'], how='outer')

In [ ]:
comparison_df

statistic relative the same, so treat as nan?

In [ ]:
mystery_df = read_csv('mystery.csv', delimiter='|', float_precision='round_trip', na_values=[-999.0])
mystery_df_train, mystery_df_test = train_test_split(mystery_df[:], train_size=0.79, random_state=seed)
mystery_df_eval, mystery_df_test = train_test_split(mystery_df_test, test_size=0.5, random_state=seed)
print(f'train: {mystery_df_train.shape}, test:{mystery_df_test.shape}, eval:{mystery_df_eval.shape}')

In [ ]:
sns.barplot(mystery_df['c'].value_counts().reset_index(), x= 'c', y= 'count')

In [ ]:
sns.scatterplot()

#### Observations on the values in each column
- x
    - all decimals
    - most positive
    - few negative
        - fixed at -999.0
        - Possible NaN?
        - Possible abnormality?
    - might be related to d2 and d3 and c
                -  when d2, d3, c, are kept constant, x is also constant
- y
    - all decimals
    - 0-13
    - few negative
        - fixed at -999.0
        - Possible NaN?
        - Possible abnormality?
- z
    - all decimals
    - most positive
    - few negative
        - fixed at -999.0
        - Possible NaN?
        - Possible abnormality?

- c
    - all decimals
    - most positive
    - no negative
    - Range 0-12 inclusive
        - except 2
        - Possible categorical column?
    - uniformly distributed
        -  unless 2 is considered
mystery = pd.read_csv(data_file, sep='|')

mystery.head()

Let's get some additional info about this dataset.

In [ ]:
mystery = mystery_df_train.replace(-999, np.nan)
mystery.isna().sum()

In [ ]:
sns.pairplot(mystery)
plt.show()

In [ ]:
sns.heatmap(mystery.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation')
plt.show()

In [ ]:
sns.pairplot(mystery, hue='c', vars=['x', 'y', 'z'])
plt.show()